In [1]:
%load_ext autoreload
%autoreload 2

from nerfstudio.model_components.nesf_components import FieldTransformerConfig, TranformerEncoderModelConfig
import torch
import lovely_tensors as lt
import time
from nerfstudio.utils.nesf_utils import visualize_point_batch
lt.monkey_patch()

/data/vision/polina/projects/wmh/dhollidt/conda/envs/nerfstudio2/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /data/vision/polina/projects/wmh/dhollidt/conda/envs/nerfstudio2/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")
/data/vision/polina/projects/wmh/dhollidt/conda/envs/nerfstudio2/lib/python3.8/site-packages/torch_points3d/core/data_transform/../../../tinycudann/modules.py:52: UserWarning: tinycudann was built for lower compute capability (75) than the system's (86). Performance may be suboptimal.
  warnings.warn(f"tinycudann was built for lower compute capability ({cc}) than the system's ({system_compute_capability}). Performance may be suboptimal.")


In [19]:
imput_size = 51
DEVICE = "cuda:0"
model_config = FieldTransformerConfig(
    knn=64,
    mode="transformer",
    transformer=TranformerEncoderModelConfig(
        num_layers=3,
        num_heads=4,
        dim_feed_forward=128,
    )
)
model = model_config.setup(input_size=imput_size)
model = model.to(DEVICE)

# parameter count
print("Parameter count: ", sum(p.numel() for p in model.parameters() if p.requires_grad))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


Parameter count:  140777


In [23]:
N=30000
Q=15000

neural_points = torch.rand(N, 3, device=DEVICE)
neural_features = torch.rand(N, imput_size, device=DEVICE, requires_grad=True)
query_points = torch.rand(Q, 3, device=DEVICE)

print("Max memory allocated: ", torch.cuda.max_memory_allocated() / 1024 / 1024, "MB")
torch.cuda.reset_peak_memory_stats()

Max memory allocated:  7.0693359375 MB


In [24]:
time1 = time.time()
K = 1

for i in range(K):
    outs = model(query_points, neural_features, neural_points)

time2 = time.time()

print("Max memory allocated: ", torch.cuda.max_memory_allocated() / 1024 / 1024, "MB")
print("Time: ", (time2 - time1) / K, "s")
print(outs.shape)
print(outs)

# reset torch stats
torch.cuda.reset_peak_memory_stats()

Querying  15000 points in neural field of  30000

Max memory allocated:  17003.8076171875 MB
Time:  0.22125887870788574 s
torch.Size([15000, 51])
tensor[15000, 51] n=765000 x∈[-2.334, 2.202] μ=0.072 σ=0.570 grad AddmmBackward0 cuda:0


In [25]:
try:
    del neural_features
except:
    pass
try:
    del outs
except:
    pass
model.zero_grad()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [8]:
torch.cuda.empty_cache()

In [ ]:
points = torch.arange(0, 1, 0.01, device=DEVICE)
points = torch.stack(torch.meshgrid(points, points, points), dim=-1).reshape(-1, 3)

ind = model.get_k_closest_points(torch.tensor([0.5, 0.5, 0.5], device=DEVICE).unsqueeze(0), points)


closest_points = points[ind]
print(closest_points)
print(closest_points.p)


tensor[1, 128, 3] all_zeros cuda:0
tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],

/data/vision/polina/projects/wmh/dhollidt/conda/envs/nerfstudio2/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
